# Check With Virus tools TotalVirus and Alienvault

## Import libaries and mount google drive and set API keys

In [ ]:
import json
import requests
from google.colab import drive
from tqdm import tqdm

drive.mount("/content/drive")

INPUT_JSON = "/content/drive/MyDrive/extracted_iocs.json"
# INPUT_JSON = "/content/drive/MyDrive/test.json"
OUTPUT_JSON = "/content/drive/MyDrive/validated_iocs.json"

VIRUSTOTAL_API_KEY = "b3a84a9a5e64d4c11c1ee585d279da328e800023c5d976101d22187f85e8ca22"
OTX_API_KEY = "67b6d1b53786bc849eafc26a05b6b6391c5684591597cd730c0f069d04baf241" # Optional

Mounted at /content/drive


## Validation

In [ ]:
with open(INPUT_JSON, "r") as f:
    data = json.load(f)


def validate_with_virustotal(ioc):
    """Check an IOC with VirusTotal"""
    url = "https://www.virustotal.com/api/v3/search"
    headers = {"x-apikey": VIRUSTOTAL_API_KEY}
    params = {"query": ioc}
    try:
        resp = requests.get(url, headers=headers, params=params, timeout=10)
        if resp.status_code == 200:
            result = resp.json()
            return result.get("data", []) != []
        else:
            return None
    except Exception as e:
        return None

def validate_with_otx(ioc):
    """Check an IOC with AlienVault OTX"""
    url = f"https://otx.alienvault.com/api/v1/indicators/IPv4/{ioc}/general"
    headers = {"X-OTX-API-KEY": OTX_API_KEY}
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        if resp.status_code == 200:
            return True
        else:
            return False
    except Exception as e:
        return None

for post in tqdm(data, desc="Validating IOCs"):
    validated = {}
    for ioc_type, ioc_list in post["regex_iocs"].items():
        validated[ioc_type] = []
        for ioc in ioc_list:
            vt_result = validate_with_virustotal(ioc)
            otx_result = validate_with_otx(ioc)
            validated[ioc_type].append({
                "ioc": ioc,
                "virustotal": vt_result,
                "otx": otx_result
            })
    post["validated_iocs"] = validated

with open(OUTPUT_JSON, "w") as f:
    json.dump(data, f, indent=2)

print(f"✅ IOC validation complete. Saved to {OUTPUT_JSON}")


Validating IOCs: 100%|██████████| 159/159 [01:51<00:00,  1.42it/s]

✅ IOC validation complete. Saved to /content/drive/MyDrive/validated_iocs.json
